
$$
\begin{aligned}
& q_{\text{ftp30}} = q_{\text{ftp0}} + 30 \times 10^{-3} \cdot \frac{1}{q_{\text{strg}} \times 10^{3}} \times \frac{180}{\pi} \times 10^{6} \\
& q_{\text{rtp30}} = q_{\text{rtp0}} - 30 \times 10^{-3} \cdot \frac{1}{q_{\text{toe}} \times 10^{3}} \times \frac{180}{\pi} \times 10^{6} \\
\end{aligned}
$$

In [ ]:

$$
\begin{aligned}
& q_{\text{ftp30}} = q_{\text{ftp0}} + 30 \times 10^{-3} \cdot \frac{1}{q_{\text{strg}} \times 10^{3}} \times \frac{180}{\pi} \times 10^{6} \\
& q_{\text{rtp30}} = q_{\text{rtp0}} - 30 \times 10^{-3} \cdot \frac{1}{q_{\text{toe}} \times 10^{3}} \times \frac{180}{\pi} \times 10^{6} \\
\end{aligned}
$$

In [1]:
import json
from ipywidgets import interactive, Button, HBox, VBox, IntSlider, FloatSlider, Output
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---unique--------------------------------
output = Output()

def interactive_plot_func(q_ftp30):
    q_ftp30_value = q_ftp30 / 180 * np.pi * 10**(-6) # [μdeg/N] --> [rad/N]
    
    q_strg = np.linspace(10, 100, 100)*10**3  # [kNm/rad] --> [Nm/rad]
    q_ftp0 = np.linspace(-100, 100, 100)/180*np.pi*10**(-6)  # [μdeg/N] --> [rad/N]

    q_strg, q_ftp0 = np.meshgrid(q_strg, q_ftp0)

    q_ftp30_calc = q_ftp0 + 30 * 10**-3 / q_strg

    fig, ax = plt.subplots()
    
    def format_q_ftp30_label(x):
        return f"q_ftp30={x*180/np.pi*10**6:.1f}"
    
    # Dataframe to save
    data = []
    
    contour = ax.contour(q_ftp0*180/np.pi*10**6, q_strg*10**(-3), q_ftp30_calc, levels=[q_ftp30_value])
    ax.clabel(contour, inline=1, fontsize=10, fmt=format_q_ftp30_label)
    for path in contour.collections[0].get_paths():
            for point in path.vertices:
                data.append({'q_ftp0': point[0], 'q_strg': point[1], 'q_ftp30': q_ftp30})


    ax.set_xlabel('Front compliance steer tp=0 $q_{ftp0}$ [μdeg/N]')
    ax.set_ylabel('Front steering stiffness $q_{strg}$ [kNm/rad]')

    ax.set_title('Front compliance steer tp=30 line')
    ax.grid()

    plt.show()

    df = pd.DataFrame(data)

    return df

def save_data(button):
    df_all = interactive_plot_func(spec_sliders["q_ftp30"].value)

    df_to_save = pd.DataFrame()

    # Extracting the q_ftp0 and q_strg values corresponding to the slider's q_ftp30 value
    df_ftp30 = df_all[['q_ftp0']].rename(columns={"q_ftp0": f"q_ftp0_q_ftp30={spec_sliders['q_ftp30'].value}"}).reset_index(drop=True)
    df_strg = df_all[['q_strg']].rename(columns={"q_strg": f"q_strg_q_ftp30={spec_sliders['q_ftp30'].value}"}).reset_index(drop=True)
    df_to_save = pd.concat([df_to_save, df_ftp30, df_strg], axis=1)

    df_to_save.to_csv('data.csv', index=False)



# ---partially unique--------------------------------

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "q_ftp30": {"name": "front compliance steer tp=30", "unit": "μdeg/N"},
}

# Creating the sliders
spec_sliders = {  
    "q_ftp30": IntSlider(min=-80, max=100, step=10, value=0, description=f'q_ftp30 [{slider_parameters["q_ftp30"]["unit"]}]', continuous_update=False),
}
condition_sliders = {
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))